In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import operator
import pandas as pd
import numpy as np

In [6]:
driver = webdriver.Firefox()
driver.implicitly_wait(3)

In [26]:
#read the datas

data_id = pd.read_csv("../data/result/freelancer_urls_indo.csv")
data_uk = pd.read_csv("../data/result/freelancer_urls_united-kingdom.csv")
data_india = pd.read_csv("../data/result/freelancer_urls_india.csv")
data_us = pd.read_csv("../data/result/freelancer_urls_united-states.csv")

In [4]:
data_id.head()

,page,link,name
0,1,https://www.upwork.com/o/profiles/users/_~01a3...,Sidiq N.
1,1,https://www.upwork.com/o/profiles/users/_~01c9...,M Alfin F.
2,1,https://www.upwork.com/o/profiles/users/_~016e...,Femmy K.
3,1,https://www.upwork.com/o/profiles/users/_~0149...,Yakub S.
4,1,https://www.upwork.com/o/profiles/users/_~0179...,Ahmad Lukman N.


In [5]:
data_uk.head()

,page,link,name
0,1,https://www.upwork.com/o/profiles/users/_~0133...,Jill S.
1,1,https://www.upwork.com/o/profiles/users/_~0103...,Agnieszka F.
2,1,https://www.upwork.com/o/profiles/users/_~0106...,Zofia R.
3,1,https://www.upwork.com/o/profiles/users/_~01e8...,Gabie L.
4,1,https://www.upwork.com/o/profiles/users/_~01fe...,Arin D.


In [5]:
data_india.head()

,page,link,name
0,1,https://www.upwork.com/o/profiles/users/~016f0...,Smriti D.
1,1,https://www.upwork.com/o/profiles/users/~01cf8...,Amit B.
2,1,https://www.upwork.com/o/profiles/users/~01bde...,Jyoti A.
3,1,https://www.upwork.com/o/profiles/users/~019f0...,Shubham R.
4,1,https://www.upwork.com/o/profiles/users/~010b6...,Sukhjot S.


In [29]:
# #try scraping one page

# try:
#     trial_link = "https://www.upwork.com/o/profiles/users/_~011e59e477bfeda05d"

#     driver.get(trial_link)


# #     hourlyrate_xpath = "//span[@itemprop='pricerange']"
# #     # freelancer_hourlyrate = WebDriverWait(driver, 10).until(
# #     #                             EC.presence_of_all_elements_located((By.XPATH, hourlyrate_xpath))
# #     #                         )
# #     freelancer_hourlyrate = driver.find_elements_by_xpath(hourlyrate_xpath)
# #     # totalearning_xpath = "/html/body/div[1]/div[2]/div/div[2]/div[2]/div/fe-profile/div[2]/div[2]/div[1]/div[1]/fe-profile-header/div/div[1]/div[2]/div[4]/ul/li[2]/h3/span"
# #     # freelancer_totalearning = WebDriverWait(driver, 10).until(
# #     #                             EC.presence_of_element_located((By.XPATH, totalearning_xpath))
# #     #                         )
# #     print(
# #         'hourly rate', freelancer_hourlyrate[0].get_attribute("innerHTML"), '\n',
# #         '\n', 'total earned', freelancer_hourlyrate[1].get_attribute("innerHTML")
# #         #'total earned', freelancer_totalearning.get_attribute("innerHTML")     
# #          )
# #     for element in freelancer_hourlyrate:
# #         print(element.get_attribute("innerHTML"))


#     location_xpath = "/html/body/div[1]/div[2]/div/div[2]/div[2]/div/fe-profile/div[2]/div[2]/div[1]/div[1]/fe-profile-header/div/div[1]/div[2]/div[1]/div[1]/div/div[2]/div[1]/fe-profile-map/span/ng-transclude/fe-profile-location-label/span[2]/span[1]/span"
#     freelancer_location = WebDriverWait(driver, 10).until(
#                                 EC.presence_of_element_located((By.XPATH, location_xpath))
#                             )
#     print('location', freelancer_location.get_attribute("innerHTML"))



# #     job_xpath = "/html/body/div[1]/div[2]/div/div[2]/div[2]/div/fe-profile/div[2]/div[2]/div[1]/div[1]/fe-profile-header/div/div[1]/div[2]/div[4]/ul/li[3]/h3"
# #     freelancer_job = WebDriverWait(driver, 10).until(
# #                             EC.presence_of_element_located((By.XPATH, job_xpath))
# #                         )
# #     workhour_xpath = "/html/body/div[1]/div[2]/div/div[2]/div[2]/div/fe-profile/div[2]/div[2]/div[1]/div[1]/fe-profile-header/div/div[1]/div[2]/div[4]/ul/li[4]/h3"
# #     freelancer_workhour = WebDriverWait(driver, 10).until(
# #                             EC.presence_of_element_located((By.XPATH, workhour_xpath))
# #                         )
# #     print(
# #         'jobs', freelancer_job.get_attribute("innerHTML"),
# #         'hours worked', freelancer_workhour.get_attribute("innerHTML")
# #     )


#     skills_class = "o-tag-skill"
#     freelancer_skills = WebDriverWait(driver, 10).until(
#                                 EC.presence_of_all_elements_located((By.CLASS_NAME, skills_class))
#                             )
#     print("skills:")
#     for skill in freelancer_skills:
#         print(skill.get_attribute("innerHTML").strip())

# except Exception as e:
#     print(e, "continue")

Message: Failed to convert data to an object
 continue


In [28]:
#get all links
links_us = data_us['link'].values.tolist()
names_us = data_us['name'].values.tolist()
freelancer_page_scrape_results = []

In [31]:
#pasang penanganan andaikan kena block dengan restart driver

count_except_skill = 0
count_except_all = 0
count_itr = 0

for i in range (0, len(names_us)):
    name = names_us[i]
    link = links_us[i]
    print(name, link, 'itr:', count_itr)
    try: 
        #go to freelancer page
        driver.get(link)

        #get freelancer location
        print("get location")
        location_xpath = "//span[@itemprop='locality']"
        freelancer_location = driver.find_element_by_xpath(location_xpath)
        freelancer_location_string = freelancer_location.get_attribute("innerHTML")

        try:
            #get freelancer skills
            print("get skills")
            skills_class = "o-tag-skill"
            freelancer_skills = WebDriverWait(driver, 3).until(
                                        EC.presence_of_all_elements_located((By.CLASS_NAME, skills_class))
                                    )
            skills_string = ''
            for idx, skill in enumerate(freelancer_skills):
                if(idx == len(freelancer_skills)-1):
                    skills_string += skill.get_attribute("innerHTML").strip()
                else:
                    skills_string += skill.get_attribute("innerHTML") + ";"

            #record result
            freelancer_page_scrape_results.append([name, link, freelancer_location_string, skills_string])
            i += 1

        except Exception as e:
            freelancer_page_scrape_results.append([name, link, freelancer_location_string, "invisible"])
            i += 1
            count_except_skill += 1
            print(e)
            print("count except skill", count_except_skill)


    except Exception as e:
        try:
            #get freelancer location
            print("get location")
            location_xpath = "//span[@itemprop='country-name']"
            freelancer_location = driver.find_element_by_xpath(location_xpath)
            freelancer_location_string = freelancer_location.get_attribute("innerHTML")

            try:
                #get freelancer skills
                print("get skills")
                skills_class = "o-tag-skill"
                freelancer_skills = WebDriverWait(driver, 7).until(
                                            EC.presence_of_all_elements_located((By.CLASS_NAME, skills_class))
                                        )
                skills_string = ''
                for idx, skill in enumerate(freelancer_skills):
                    if(idx == len(freelancer_skills)-1):
                        skills_string += skill.get_attribute("innerHTML").strip()
                    else:
                        skills_string += skill.get_attribute("innerHTML") + ";"

                #record result
                freelancer_page_scrape_results.append([name, link, freelancer_location_string, skills_string])
                i += 1

            except Exception as e:
                freelancer_page_scrape_results.append([name, link, freelancer_location_string, "invisible"])
                i += 1
                count_except_skill += 1
                print(e)
                print("count except skill", count_except_skill)

        except:
            try:
                driver.find_element_by_xpath("/html/body/section/div[2]/div/h1")
                driver = webdriver.Firefox()
                driver.implicitly_wait(5)
                print("new driver")
                count_itr = 0
                count_except_skill = 0
                count_except_all = 0
            except:
                freelancer_page_scrape_results.append([name, link, "none", "none"])
                i += 1
                count_except_all += 1
                print(e)
                print("count except all", count_except_all)

    count_itr += 1

Brian M M. https://www.upwork.com/o/profiles/users/~010f13f69ef761dda7 itr: 0
get location
get skills
Robert W. https://www.upwork.com/o/profiles/users/~018b5ce44849fd24d7 itr: 1
get location
get skills
Jami H. https://www.upwork.com/o/profiles/users/~012d70da5f7ecbedb2 itr: 2
get location
get skills
Michael P. https://www.upwork.com/o/profiles/users/~0132d5accdad6707f7 itr: 3
get location
get skills
Jack M. https://www.upwork.com/o/profiles/users/~016a1667200a0b8cb0 itr: 4
get location
get skills
David T. https://www.upwork.com/o/profiles/users/~01c407f91b187afa7d itr: 5
get location
get skills
Ewan Ellis M. https://www.upwork.com/o/profiles/users/~01db74ce3e7227751d itr: 6
get location
get skills
Andrew H. https://www.upwork.com/o/profiles/users/~01fab9a961adfd18a9 itr: 7
get location
get skills
Melanie S. https://www.upwork.com/o/profiles/users/~0175c2eb4473099f31 itr: 8
get location
get skills
Laura K. https://www.upwork.com/o/profiles/users/~01d7d072a2ee328490 itr: 9
get location


In [32]:
freelancer_data_results = pd.DataFrame(freelancer_page_scrape_results, columns=['name', 'link', 'location', 'skills'])
freelancer_data_results.drop_duplicates(subset='link', keep='first', inplace=True)
freelancer_data_results.head()

,name,link,location,skills
0,Brian M M.,https://www.upwork.com/o/profiles/users/~010f1...,chesapeake,\nPHP\n<!-- ngIf: isSkillChecked(skill) -->\n;...
1,Robert W.,https://www.upwork.com/o/profiles/users/~018b5...,woodbury,Writing\n<!-- ngIf: isSkillChecked(skill) -->
2,Jami H.,https://www.upwork.com/o/profiles/users/~012d7...,saint joseph,\nGeneral Profile\n;\nBusiness Development\n<!...
3,Michael P.,https://www.upwork.com/o/profiles/users/~0132d...,new york,\nGeneral Profile\n;\nSearch Engine Optimizati...
4,Jack M.,https://www.upwork.com/o/profiles/users/~016a1...,lorain,\nNegotiation\n<!-- ngIf: isSkillChecked(skill...


In [33]:
print(freelancer_data_results.shape)

(4989, 4)


In [35]:
freelancer_data_results[freelancer_data_results['location'] == 'none']

,name,link,location,skills
1532,Lauren C.,https://www.upwork.com/o/profiles/users/~01132...,none,none
2225,Haritha R.,https://www.upwork.com/o/profiles/users/~01c17...,none,none
2523,Julian C.,https://www.upwork.com/o/profiles/users/~0179d...,none,none
2534,Shundalyn A.,https://www.upwork.com/o/profiles/users/~01d3e...,none,none
2772,Alexander K.,https://www.upwork.com/o/profiles/users/~01278...,none,none
3495,Leonardo F.,https://www.upwork.com/o/profiles/users/~01009...,none,none
4321,Deidra P.,https://www.upwork.com/o/profiles/users/~01a79...,none,none
4333,Brandon S.,https://www.upwork.com/o/profiles/users/~01965...,none,none
4813,Guangming L.,https://www.upwork.com/o/profiles/users/~01b00...,none,none


In [34]:
assert 0 == 1
COUNTRY_NAME = "us"
freelancer_data_results.to_csv("../data/result/freelancer_data_results_"+COUNTRY_NAME+".csv")

In [21]:
#assert 0 == 1
#impute those who are still none
names_uk = freelancer_data_results.loc[freelancer_data_results['location'] == 'none', 'name'].values.tolist()
links_uk = freelancer_data_results.loc[freelancer_data_results['location'] == 'none', 'link'].values.tolist()
freelancer_page_scrape_results = []

driver = webdriver.Firefox()
driver.implicitly_wait(15)

In [22]:
#impute those who are still none
#pasang penanganan andaikan kena block dengan restart driver

count_except_skill = 0
count_except_all = 0
count_itr = 0

for name, link in zip(names_uk, links_uk):
    print(name, link, 'itr:', count_itr)
    try: 
        #go to freelancer page
        driver.get(link)

        #get freelancer location
        print("get location")
        location_xpath = "//span[@itemprop='locality']"
        freelancer_location = driver.find_element_by_xpath(location_xpath)
        freelancer_location_string = freelancer_location.get_attribute("innerHTML")

        try:
            #get freelancer skills
            print("get skills")
            skills_class = "o-tag-skill"
            freelancer_skills = WebDriverWait(driver, 3).until(
                                        EC.presence_of_all_elements_located((By.CLASS_NAME, skills_class))
                                    )
            skills_string = ''
            for idx, skill in enumerate(freelancer_skills):
                if(idx == len(freelancer_skills)-1):
                    skills_string += skill.get_attribute("innerHTML").strip()
                else:
                    skills_string += skill.get_attribute("innerHTML") + ";"

            #record result
            freelancer_page_scrape_results.append([name, link, freelancer_location_string, skills_string])

        except Exception as e:
            freelancer_page_scrape_results.append([name, link, freelancer_location_string, "invisible"])
            count_except_skill += 1
            print(e)
            print("count except skill", count_except_skill)


    except Exception as e:
        try:
            #get freelancer location
            print("get location")
            location_xpath = "//span[@itemprop='country-name']"
            freelancer_location = driver.find_element_by_xpath(location_xpath)
            freelancer_location_string = freelancer_location.get_attribute("innerHTML")

            try:
                #get freelancer skills
                print("get skills")
                skills_class = "o-tag-skill"
                freelancer_skills = WebDriverWait(driver, 10).until(
                                            EC.presence_of_all_elements_located((By.CLASS_NAME, skills_class))
                                        )
                skills_string = ''
                for idx, skill in enumerate(freelancer_skills):
                    if(idx == len(freelancer_skills)-1):
                        skills_string += skill.get_attribute("innerHTML").strip()
                    else:
                        skills_string += skill.get_attribute("innerHTML") + ";"

                #record result
                freelancer_page_scrape_results.append([name, link, freelancer_location_string, skills_string])

            except Exception as e:
                freelancer_page_scrape_results.append([name, link, freelancer_location_string, "invisible"])
                count_except_skill += 1
                print(e)
                print("count except skill", count_except_skill)

        except:
            try:
                driver.find_element_by_xpath("/html/body/section/div[2]/div/h1")
                driver = webdriver.Firefox()
                driver.implicitly_wait(5)
                print("new driver")
                count_itr = 0
                count_except_skill = 0
                count_except_all = 0
            except:
                freelancer_page_scrape_results.append([name, link, "none", "none"])
                count_except_all += 1
                print(e)
                print("count except all", count_except_all)

    count_itr += 1

Kumar P. https://www.upwork.com/o/profiles/users/~01b22150ddb66a44a9 itr: 0
get location
get skills
Message: 

count except skill 1
Sunanda S. https://www.upwork.com/o/profiles/users/~0137daa1086a539bc1 itr: 1
get location
get skills
Message: 

count except skill 2
Harminder K. https://www.upwork.com/o/profiles/users/~0161171d1f980bef9d itr: 2
get location
get skills
Roushan K. https://www.upwork.com/o/profiles/users/~01f65649a02a07af16 itr: 3
get location
get skills
Rohit J. https://www.upwork.com/o/profiles/users/~01f83994484f65a07c itr: 4
get location
get skills
Message: 

count except skill 3
Rajat V. https://www.upwork.com/o/profiles/users/~01698d2f6ab07cac0d itr: 5
get location
get skills
Jugal K. https://www.upwork.com/o/profiles/users/~0162fbb179c09516a2 itr: 6
get location
get location
Message: Unable to locate element: //span[@itemprop='locality']

count except all 1
Nitish K. https://www.upwork.com/o/profiles/users/~01e66fcbc56b1f575c itr: 7
get location
get skills
Message: 

In [23]:
original_data = pd.read_csv("../data/result/freelancer_data_results_india.csv")
temp_data = pd.DataFrame(freelancer_page_scrape_results, columns=['name', 'link', 'location', 'skills'])
scraped_links = temp_data['link'].values.tolist()
not_scraped = original_data[~original_data['link'].isin(scraped_links)]
new_data = not_scraped.append(temp_data)
new_data.drop_duplicates(subset='link', keep='first', inplace=True)

In [24]:
#find out how many nones ar left
new_data[new_data['location'] == 'none'].shape

(1, 5)

In [25]:
assert 0 == 1
new_data.to_csv("../data/result/freelancer_data_results_"+COUNTRY_NAME+".csv", index=False)

In [91]:
names_uk = new_data.loc[new_data['location'] == 'none', 'name'].values.tolist()
links_uk = new_data.loc[new_data['location'] == 'none', 'link'].values.tolist()
freelancer_page_scrape_results = []

driver = webdriver.Firefox()
driver.implicitly_wait(2)

In [92]:
#impute those who are still none
#pasang penanganan andaikan kena block dengan restart driver

count_except_skill = 0
count_except_all = 0
count_itr = 0

for name, link in zip(names_uk, links_uk):
    print(name, link, 'itr:', count_itr)
    try: 
        #go to freelancer page
        driver.get(link)

        #get freelancer location
        print("get location")
        location_xpath = "//span[@itemprop='locality']"
        freelancer_location = driver.find_element_by_xpath(location_xpath)
        freelancer_location_string = freelancer_location.get_attribute("innerHTML")

        try:
            #get freelancer skills
            print("get skills")
            skills_class = "o-tag-skill"
            freelancer_skills = WebDriverWait(driver, 3).until(
                                        EC.presence_of_all_elements_located((By.CLASS_NAME, skills_class))
                                    )
            skills_string = ''
            for idx, skill in enumerate(freelancer_skills):
                if(idx == len(freelancer_skills)-1):
                    skills_string += skill.get_attribute("innerHTML").strip()
                else:
                    skills_string += skill.get_attribute("innerHTML") + ";"

            #record result
            freelancer_page_scrape_results.append([name, link, freelancer_location_string, skills_string])

        except Exception as e:
            freelancer_page_scrape_results.append([name, link, freelancer_location_string, "invisible"])
            count_except_skill += 1
            print(e)
            print("count except skill", count_except_skill)


    except Exception as e:
        try:
            #get freelancer location
            print("get location")
            location_xpath = "//span[@itemprop='country-name']"
            freelancer_location = driver.find_element_by_xpath(location_xpath)
            freelancer_location_string = freelancer_location.get_attribute("innerHTML")

            try:
                #get freelancer skills
                print("get skills")
                skills_class = "o-tag-skill"
                freelancer_skills = WebDriverWait(driver, 10).until(
                                            EC.presence_of_all_elements_located((By.CLASS_NAME, skills_class))
                                        )
                skills_string = ''
                for idx, skill in enumerate(freelancer_skills):
                    if(idx == len(freelancer_skills)-1):
                        skills_string += skill.get_attribute("innerHTML").strip()
                    else:
                        skills_string += skill.get_attribute("innerHTML") + ";"

                #record result
                freelancer_page_scrape_results.append([name, link, freelancer_location_string, skills_string])

            except Exception as e:
                freelancer_page_scrape_results.append([name, link, freelancer_location_string, "invisible"])
                count_except_skill += 1
                print(e)
                print("count except skill", count_except_skill)

        except:
            try:
                driver.find_element_by_xpath("/html/body/section/div[2]/div/h1")
                driver = webdriver.Firefox()
                driver.implicitly_wait(5)
                print("new driver")
                count_itr = 0
                count_except_skill = 0
                count_except_all = 0
            except:
                freelancer_page_scrape_results.append([name, link, "none", "none"])
                count_except_all += 1
                print(e)
                print("count except all", count_except_all)

    count_itr += 1

Thiago S. https://www.upwork.com/o/profiles/users/_~01e8f87ed3f00f6926 itr: 0
get location
get location
Message: Unable to locate element: //span[@itemprop='locality']

count except all 1
Steve M. https://www.upwork.com/o/profiles/users/_~013a58baf7e3adeb74 itr: 1
get location
get location
Message: Unable to locate element: //span[@itemprop='locality']

count except all 2
Behroz A. https://www.upwork.com/o/profiles/users/_~0178a2fcf518190ee2 itr: 2
get location
get location
Message: Unable to locate element: //span[@itemprop='locality']

count except all 3
Louise B. https://www.upwork.com/o/profiles/users/_~010c3c61d6cfbe3c97 itr: 3
get location
get location
Message: Unable to locate element: //span[@itemprop='locality']

count except all 4
Elif Gumruk M. https://www.upwork.com/o/profiles/users/_~011e59e477bfeda05d itr: 4
get location
get location
Message: Unable to locate element: //span[@itemprop='locality']

count except all 5
Katherine F. https://www.upwork.com/o/profiles/users/_~0